# Ultrasound Nerve Segmentation

## Identify nerve structures in ultrasound images of the neck

<https://www.kaggle.com/c/ultrasound-nerve-segmentation>

> Even the bravest patient cringes at the mention of a surgical procedure. Surgery inevitably brings discomfort, and oftentimes involves significant post-surgical pain. Currently, patient pain is frequently managed through the use of narcotics that bring a bevy of unwanted side effects.
This competition's sponsor is working to improve pain management through the use of indwelling catheters that block or mitigate pain at the source. Pain management catheters reduce dependence on narcotics and speed up patient recovery.
**Accurately identifying nerve structures in ultrasound images is a critical step in effectively inserting a patient’s pain management catheter.** In this competition, Kagglers are challenged to build a model that can identify nerve structures in a dataset of ultrasound images of the neck. Doing so would improve catheter placement and contribute to a more pain free future. 

## Data

In [ ]:
import os
import numpy as np
from skimage.io import imsave, imread
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
data_path = '../raw/'

image_rows = 420
image_cols = 580

### Load the training images

In [ ]:
train_data_path = os.path.join(data_path, 'train')
images = os.listdir(train_data_path)
total = len(images) // 2

In [ ]:
imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
imgs_mask = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)

i = 0
for image_name in tqdm_notebook(images):    
    if 'mask' in image_name:
        continue
        
    image_mask_name = image_name.split('.')[0] + '_mask.tif'
    img = imread(os.path.join(train_data_path, image_name), as_grey=True)
    img_mask = imread(os.path.join(train_data_path, image_mask_name), as_grey=True)

    img = np.array([img])
    img_mask = np.array([img_mask])

    imgs[i] = img
    imgs_mask[i] = img_mask
    i += 1

In [ ]:
f = plt.figure(figsize=(8, 14))
for i in range(4):
    sp = f.add_subplot(4, 2, 2*i+1)
    sp.axis('Off')
    plt.imshow(imgs[i, :, :], cmap="gray")
    sp = f.add_subplot(4, 2, 2*i+2)
    sp.axis('Off')
    plt.imshow(imgs_mask[i, :, :], cmap="gray")

### Load the test images

In [ ]:
test_data_path = os.path.join(data_path, 'test')
images = os.listdir(test_data_path)
total = len(images)

imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
imgs_id = np.ndarray((total, ), dtype=np.int32)

i = 0
for image_name in tqdm_notebook(images):
    img_id = int(image_name.split('.')[0])
    img = imread(os.path.join(test_data_path, image_name), as_grey=True)

    img = np.array([img])

    imgs[i] = img
    imgs_id[i] = img_id
    i += 1

In [ ]:
f = plt.figure(figsize=(8, 6))
for i in range(4):
    sp = f.add_subplot(2, 2, i+1)
    sp.axis('Off')
    sp.set_title(f'ID: {imgs_id[i]}')
    plt.imshow(imgs[i, :, :], cmap="gray")

## Training a deep neural network

In [ ]:
from skimage.transform import resize
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [ ]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
img_rows = 96
img_cols = 96
smooth = 1.

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [ ]:
imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255.  # scale masks to [0, 1]

In [ ]:
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

In [ ]:
model.summary()

In [ ]:
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

model.fit(imgs_train, imgs_mask_train, batch_size=32, nb_epoch=20, verbose=1, shuffle=True,
          validation_split=0.2,
          callbacks=[model_checkpoint])

In [ ]:
imgs_test = preprocess(imgs_test)

imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

In [ ]:
model.load_weights('weights.h5')

imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save('imgs_mask_test.npy', imgs_mask_test)

pred_dir = 'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, imgs_id_test):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

In [ ]:
def prep(img):
    img = img.astype('float32')
    img = (img > 0.5).astype(np.uint8)  # threshold
    img = resize(img, (image_cols, image_rows), preserve_range=True)
    return img


def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])

In [ ]:
argsort = np.argsort(imgs_id_test)
imgs_id_test = imgs_id_test[argsort]
imgs_test = imgs_test[argsort]

total = imgs_test.shape[0]
ids = []
rles = []
for i in tqdm_notebook(range(total)):
    img = imgs_test[i, 0]
    img = prep(img)
    rle = run_length_enc(img)

    rles.append(rle)
    ids.append(imgs_id_test[i])

first_row = 'img,pixels'
file_name = 'submission.csv'

with open(file_name, 'w+') as f:
    f.write(first_row + '\n')
    for i in range(total):
        s = str(ids[i]) + ',' + rles[i]
        f.write(s + '\n')